<a href="https://colab.research.google.com/github/yohaankarian/LoanFit-Machine-Learning-based-Loan-Approval-System/blob/main/LoanFit%20-%20TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Supressing the warning messages
import warnings
warnings.filterwarnings('ignore')

In [2]:
#reading the dataset
import pandas as pd
import numpy as np

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Loan_Approval_Data.csv to Loan_Approval_Data.csv


In [4]:
import io
Loandata = pd.read_csv(io.BytesIO(uploaded['Loan_Approval_Data.csv']))

In [5]:
#Removing useless data from the data
#Loan_ID is not useful in the predictive analysis because these variables are not qualitative
UselessColumns = ['Loan_ID']
Loandata = Loandata.drop(UselessColumns,axis=1)
Loandata.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [6]:
# Finding nearest values to 30000 mark for applicant income
Loandata['ApplicantIncome'][Loandata['ApplicantIncome']>20000].sort_values()

130    20166
308    20233
284    20667
506    20833
126    23803
183    33846
443    37719
185    39147
155    39999
171    51763
333    63337
409    81000
Name: ApplicantIncome, dtype: int64

In [7]:
Loandata['ApplicantIncome'][Loandata['ApplicantIncome']>30000] = 23803

In [8]:
# Replacing outliers with nearest possibe value
Loandata['CoapplicantIncome'][Loandata['CoapplicantIncome']>15000] = 11300

In [9]:
# Replacing the missing values
# Using MODE for categorical columns
Loandata['Gender'].fillna(Loandata['Gender'].mode()[0], inplace=True)
Loandata['Married'].fillna(Loandata['Married'].mode()[0], inplace=True)
Loandata['Dependents'].fillna(Loandata['Dependents'].mode()[0], inplace=True)
Loandata['Self_Employed'].fillna(Loandata['Self_Employed'].mode()[0], inplace=True)
# Using Mode value for Loan_Amount_Term since it is a categorical variable
Loandata['Loan_Amount_Term'].fillna(Loandata['Loan_Amount_Term'].mode()[0], inplace=True)
Loandata['Credit_History'].fillna(Loandata['Credit_History'].mode()[0], inplace=True)

# Using Median value for continuous columns
Loandata['LoanAmount'].fillna(Loandata['LoanAmount'].median(), inplace=True)

In [10]:
SelectedColumns=['Married', 'Education', 'Credit_History', 'Property_Area']

# Selecting final columns
DataForML=Loandata[SelectedColumns]
DataForML.head()

,Married,Education,Credit_History,Property_Area
0,No,Graduate,1.0,Urban
1,Yes,Graduate,1.0,Rural
2,Yes,Graduate,1.0,Urban
3,Yes,Not Graduate,1.0,Urban
4,No,Graduate,1.0,Urban


In [11]:
from sklearn.model_selection import train_test_split

In [13]:
X = pd.get_dummies(DataForML)
y = Loandata['Loan_Status'].apply(lambda x: 1 if x == 'Y' else 0)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = .3)

In [15]:
X_train.head()

,Credit_History,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
339,1.0,1,0,1,0,0,1,0
231,1.0,0,1,1,0,1,0,0
303,1.0,0,1,1,0,0,0,1
426,1.0,1,0,0,1,1,0,0
73,0.0,0,1,0,1,0,1,0


In [16]:
y_train.head()

339    1
231    1
303    1
426    0
73     0
Name: Loan_Status, dtype: int64

In [18]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

In [19]:
model = Sequential()
model.add(Dense(units = 8,activation = 'relu', input_dim = len(X_train.columns)))
model.add(Dense(units = 8, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))


In [21]:
model.compile(loss='binary_crossentropy', optimizer = 'sgd', metrics = 'accuracy')# sgd - stochastic gradient

In [22]:
model.fit(X_train,y_train, epochs = 500, batch_size = 32)

Epoch 1/500
14/14 [==============================] - 1s 2ms/step - loss: 0.6668 - accuracy: 0.5967
Epoch 2/500
14/14 [==============================] - 0s 2ms/step - loss: 0.6451 - accuracy: 0.7156
Epoch 3/500
14/14 [==============================] - 0s 2ms/step - loss: 0.6280 - accuracy: 0.7063
Epoch 4/500
14/14 [==============================] - 0s 2ms/step - loss: 0.6147 - accuracy: 0.6900
Epoch 5/500
14/14 [==============================] - 0s 2ms/step - loss: 0.6020 - accuracy: 0.6853
Epoch 6/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5931 - accuracy: 0.6853
Epoch 7/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5860 - accuracy: 0.6853
Epoch 8/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5815 - accuracy: 0.6853
Epoch 9/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5784 - accuracy: 0.6853
Epoch 10/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5762 - accuracy: 0.6853
Epoch 11/

In [23]:
#Prediction:
y_hat = model.predict(X_test)
y_hat = [0 if val < 0.5 else 1 for val in y_hat]

6/6 [==============================] - 0s 2ms/step


In [24]:
y_hat

[1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [25]:
accuracy_score(y_test, y_hat)

0.8

In [26]:
model.save('tfmodel')